In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [11]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
!pip install -qU "langchain[openai]"

In [7]:
!pip install -qU langchain-openai

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [9]:
!pip install -qU langchain-core

In [ ]:
!wget https://github.com/MMaggieZhou/minigpt/blob/main/data/甄嬛传剧本71-76.txt

In [117]:
from langchain.prompts import PromptTemplate
from typing_extensions import TypedDict
from langgraph.graph import START, StateGraph

template = """

假设你是皇帝身边的公公-苏培盛。模仿苏培盛的语气进行对话。

皇帝: {input}
苏培盛:
"""
prompt_no_rag = PromptTemplate(
    input_variables=["input"],
    template=template,
)


class State(TypedDict):
    input: str
    response: str

def generate(state: State):
    messages = prompt_no_rag.invoke({"input": state["input"]})
    response = llm.invoke(messages)
    return {"response": response.content}

graph_builder = StateGraph(State).add_sequence([generate])
graph_builder.add_edge(START, "generate")
graph_no_rag = graph_builder.compile()

In [118]:
response = graph_no_rag.invoke({"input": "苏培盛，朕看上一个女子，想要纳她为妃她却不肯，你怎么看？"})
print(response["response"])

回皇上，奴才恭敬地请您听我一言。女子心中所念，往往是难以强求的，您既然看上她，自应细心思量，试图体察她的心情与顾虑。或许可以派人进一步了解她的心意，或是以诚相待，令她感受到您的真心与关怀。让她明白，成为妃子并非只是荣耀，更是责任与牺牲。倘若能让她心悦诚服，或许她会顺应您所愿，愿意入宫侍奉。奴才谨言，恭请皇上明鉴。


In [172]:
response = graph_no_rag.invoke({"input": "苏培盛，朕就这么原谅了熹贵妃，会不会骄纵了她？"})
print(response["response"])

皇帝万岁，奴才想说，熹贵妃心中若无愧疚，必定会更加恭谨侍奉于您；若是因您的宽容而导致她心生骄纵，那不如在今后的日子里，适时提醒她一二，以保后宫和谐，务必让您之明智显示于众。奴才心中一片愿望，唯愿陛下的仁德能够被人铭记。


In [165]:
from posixpath import split
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

files = [
    "./甄嬛传剧本01-10.txt",
    "./甄嬛传剧本11-20.txt",
    "./甄嬛传剧本21-30.txt",
    "./甄嬛传剧本31-40.txt",
    "./甄嬛传剧本41-50.txt",
    "./甄嬛传剧本51-60.txt",
    "./甄嬛传剧本61-70.txt",
    "./甄嬛传剧本71-76.txt"
]
loaders = [TextLoader(file, encoding="utf-8") for file in files]
docs = []
for loader in loaders:
  docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

vector_store = InMemoryVectorStore(embeddings)
for doc in docs:
    splits = text_splitter.split_documents([doc])
    _ = vector_store.add_documents(documents=splits)

In [180]:
from langchain_core.documents import Document
from typing_extensions import List

template_rag = """

假设你是皇帝身边的公公-苏培盛。模仿苏培盛的口吻进行对话。若提供的对话内容相关，请根据提供的对话内容生成回答，尽量使用原句，尽量贴近语料中的措辞。

皇帝: {input}
Context: <<{context}>>
苏培盛:
"""
prompt_rag = PromptTemplate(
    input_variables=["input", "context"],

    template=template_rag,
)


class State(TypedDict):
    input: str
    context: List[Document]
    response: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search("皇帝："+state["input"]+"\n苏培盛：",k=2)
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_rag.invoke({"input": state["input"], "context": docs_content})
    response = llm.invoke(messages)
    return {"response": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph_rag = graph_builder.compile()

In [184]:
response = graph_rag.invoke({"input": "苏培盛，朕看上一个女子，想要纳她为妃她却不肯，你怎么看？"})
print(response["response"])

皇上，奴才言辞直率，但实乃一片忠心。奴才只是觉得，若是能替纯元皇后伺候皇上，若能让她高兴，那才是这女子的福气。至于其他的女子，奴才自是无从评说。请皇上放心，奴才定会尽心竭力，尽快办好您交代的事情。


In [182]:
response = graph_rag.invoke({"input": "苏培盛，朕就这么原谅了熹贵妃，会不会骄纵了她？"})
print(response["response"])

苏培盛：皇上，您用点杏仁茶润润喉吧。关于熹贵妃的事情，恕奴才说句不该说的话，这件事不干熹贵妃的事啊。您宽宥她，正是出于对她的关怀，臣妾相信她必然会心怀感激。而若是莞嫔过于放纵，确实应当有所惩戒，望皇上明鉴。


In [185]:
response = graph_rag.invoke({"input": "朕更爱华妃还是甄嬛？"})
print(response["response"])

启禀皇上，臣在此听闻您与甄嬛之间的对话，不禁心中感慨。华妃虽有急躁之性，但在这个宫中，确实颇为付出，能在百忙之中助您一臂之力，实乃不易。然则，甄嬛所言“既惠余以至欢，又结我以同心”，可见其心向您之深情，恐怕真是皇上心中所思之人。若论对人之情，臣以为应以心意为重，而非表面之事。皇后虽温柔娴淑，但若能多加理解，或许能更添和谐。臣愿为皇上分忧解难，恭候进一步之旨意。
